# (1) Bars functions

## (1.1) Tick Bar

In [229]:
class T_bar():

    # this class of functions is used in the calculation and creating the volume  
    # imbalance bars

    def __init__(self):
        self.timestamp = None
        self.open = None
        self.high = None
        self.low = None
        self.close = None
        self.volume = None
        self.nticks = None
    
    def initiate_bar(self,row):
        price = round(row.price,2)
        
        self.timestamp = row.timestamp
        self.open = price
        self.high = price
        self.low = price
        self.close = price
        self.volume = round(row.qty,6)
        self.nticks = 1
        
    def update(self,row):

        price = round(row.price,2)
        
        if self.high < price:
            self.high = price

        elif self.low > price:
            self.low = price

        self.close = price
        self.volume += round(row.qty,6)
        self.nticks += 1
        
    def get_nticks(self):
        return self.nticks
        
    def create_bar(self):
        
        bar = {
        'timestamp': self.timestamp,        # Start time
        'open': self.open,
        'high': self.high,
        'low': self.low,
        'close': self.close,
        'volume': self.volume,
        'E_T':self.nticks,
        #"vwap": round((df['price'] * df['amount']).sum() / df['amount'].sum(),2)
        }
        
        # Convert to DataFrame (optional)
        bar_df = pd.DataFrame([bar])
        # Output
        return bar_df    

## (1.2) Volume Bar

In [230]:
class V_bar():

    # this class of functions is used in the calculation and creating the volume  
    # imbalance bars

    def __init__(self):
        self.timestamp = None
        self.open = None
        self.high = None
        self.low = None
        self.close = None
        self.volume = None
        self.nticks = None
    
    def initiate_bar(self,row):
        price = round(row.price,2)
        
        self.timestamp = row.timestamp
        self.open = price
        self.high = price
        self.low = price
        self.close = price
        self.volume = round(row.qty,6)
        self.nticks = 1
        
    def update(self,row):

        price = round(row.price,2)
        
        if self.high < price:
            self.high = price

        elif self.low > price:
            self.low = price

        self.close = price
        self.volume += round(row.qty,6)
        self.nticks += 1
        
    def get_total_volume(self):
        return self.volume
        
    def create_bar(self):
        
        bar = {
        'timestamp': self.timestamp,        # Start time
        'open': self.open,
        'high': self.high,
        'low': self.low,
        'close': self.close,
        'volume': self.volume,
        'E_T':self.nticks,
        #"vwap": round((df['price'] * df['amount']).sum() / df['amount'].sum(),2)
        }
        
        # Convert to DataFrame (optional)
        bar_df = pd.DataFrame([bar])
        # Output
        return bar_df

## (1.3) Volume Imbalance Bar

In [231]:
class VI_bar():

    # this class of functions is used in the calculation and creating the volume  
    # imbalance bars

    def __init__(self):
        self.timestamp = None
        self.open = None
        self.high = None
        self.low = None
        self.close = None
        self.volume = None
        self.total_signed_vol = None
        self.nticks = None
    
    def initiate_bar(self,row):
        price = round(row.price,2)
        
        self.timestamp = row.timestamp
        self.open = price
        self.high = price
        self.low = price
        self.close = price
        self.volume = round(row.qty,6)
        self.total_signed_vol = round(row.signed_vol,6)
        self.nticks = 1
        
    def update(self,row):

        price = round(row.price,2)
        
        if self.high < price:
            self.high = price

        elif self.low > price:
            self.low = price

        self.close = price
        self.volume += round(row.qty,6)
        self.total_signed_vol += round(row.signed_vol,6)
        self.nticks += 1
        
    def get_total_signed_vol(self):
        return self.total_signed_vol

    def get_nticks(self):
        return self.nticks
    
    def create_bar(self):
        
        bar = {
        'timestamp': self.timestamp,        # Start time
        'open': self.open,
        'high': self.high,
        'low': self.low,
        'close': self.close,
        'volume': self.volume,
        'E_T':self.nticks,
        #"vwap": round((df['price'] * df['amount']).sum() / df['amount'].sum(),2)
        }
        
        # Convert to DataFrame (optional)
        bar_df = pd.DataFrame([bar])
        # Output
        return bar_df

## (1.4) Calculate the properties

In [221]:
class predictors():

    def Relative_strength_index(self):
        n = 14
        
        average_gain= np.full((1,n-1),np.nan)
        average_loss= np.full((1,n-1),np.nan)
    
        for t in range(n,len(self.changes)+1):
            interval = np.array(self.changes[t-n:t])
    
            gain = np.array([])
            loss = np.array([])
    
            for i in interval:
                if (i> 0):
                    gain = np.append(gain,i)
                elif(i< 0): 
                    loss = np.append(loss,i)
            
            # average gain
            try:
                average_gain = np.append(average_gain,mean(gain))
            except:
                average_gain = np.append(average_gain,1)
    
            # average loss
            try:
                average_loss = np.append(average_loss,mean(abs(loss)))
            except:
                average_loss = np.append(average_loss,1) 
    
        RS = np.array(average_gain/average_loss)
        RSI = np.array(100 - (100/(1+RS)))

        res = np.append(np.full((1,1),np.nan),RSI)
        return(res)

    def Average_true_range(self):

        # True Range components
        high_low = self.data['high'] - self.data['low']
        high_close_prev = (self.data['high'] - self.data['close'].shift(1)).abs()
        low_close_prev = (self.data['low'] - self.data['close'].shift(1)).abs()
        
        # True Range
        tr = pd.concat([high_low, high_close_prev, low_close_prev], axis=1).max(axis=1)
        
        # ATR (simple moving average of TR)
        atr = tr.rolling(window=14, min_periods=1).mean()
        
        return atr
    
    def Average_directional_index(self):
        
         # 1. Calculate True Range (TR)
        high_low = self.data['high'] - self.data['low']
        high_close_prev = (self.data['high'] - self.data['close'].shift(1)).abs()
        low_close_prev = (self.data['low'] - self.data['close'].shift(1)).abs()
        tr = pd.concat([high_low, high_close_prev, low_close_prev], axis=1).max(axis=1)
    
        # 2. Calculate directional movements
        plus_dm = self.data['high'] - self.data['high'].shift(1)
        minus_dm = self.data['low'].shift(1) - self.data['low']
    
        plus_dm = np.where((plus_dm > minus_dm) & (plus_dm > 0), plus_dm, 0.0)
        minus_dm = np.where((minus_dm > plus_dm) & (minus_dm > 0), minus_dm, 0.0)
    
        # 3. Smooth TR, +DM, -DM (Wilder's smoothing)
        tr_smooth = pd.Series(tr).rolling(window=14).sum()
        plus_dm_smooth = pd.Series(plus_dm).rolling(window=14).sum()
        minus_dm_smooth = pd.Series(minus_dm).rolling(window=14).sum()
    
        # 4. Calculate +DI and -DI
        plus_di = 100 * (plus_dm_smooth / tr_smooth)
        minus_di = 100 * (minus_dm_smooth / tr_smooth)
    
        # 5. Calculate DX
        dx = (abs(plus_di - minus_di) / (plus_di + minus_di)) * 100
    
        # 6. ADX = smoothed DX
        adx = dx.rolling(window=14).mean()
    
        # Return as DataFrame
        df_out = pd.DataFrame()
        df_out['+DI'] = plus_di
        df_out['-DI'] = minus_di
        df_out['ADX'] = adx
    
        return df_out
    
    def __init__(self,data):

        self.data = data
        
        #### changes from price #####
        self.changes = np.array([])
        close = np.array(data["close"])
        for t in range(1,len(close)):
            self.changes = np.append(self.changes,(close[t]-close[t-1]))

        ### moving averages ###
        SMA_9 = np.array(data["close"].rolling(window=9).mean())
        SMA_21 = np.array(data["close"].rolling(window=21).mean())
        EMA_9 = np.array(data["close"].ewm(span=9, adjust=False, min_periods=9).mean())
        EMA_21 = np.array(data["close"].ewm(span=21, adjust=False, min_periods=9).mean())

        ### Relative Strength Index (RSI) (14) ###
        RSI = self.Relative_strength_index()

        ### ATR (Average True Range) (14) ###
        ATR = self.Average_true_range()
        
        ### ADX (Average Directional Index) (14) ###
        ADX = self.Average_directional_index()
        
        ### predictors ###
        # "MiddleBand":BD["MiddleBand"],"UpperBand":BD["UpperBand"],"LowerBand":BD["LowerBand"],"MACD":MACD,"ATR":ATR,"ADX":ADX, 
        self.predictors = {"timestamp":self.data["timestamp"],"SMA 9":np.round(SMA_9,2),"SMA 21":np.round(SMA_21,2),"EMA 9":np.round(EMA_9,2),"EMA 21":np.round(EMA_21,2),
                             "RSI":np.round(RSI,2),"ATR":np.round(ATR,2),"+DI":np.round(ADX["+DI"],2),"-DI":np.round(ADX["-DI"],2),"ADX":np.round(ADX["ADX"],2)}
        
    def get_predictors(self):
        return(self.predictors)

# (2) Data Preprocessing / Cleaning
This code is used to:
- Data Loading
- Data Preprocessing / Cleaning:
    - Calculate standard bars (Tick bars), based in minute statistics for the thresholds, calculate exponential the moving averages for tresholds.
    - Calculate standard bars (Volume bars), based in minute statistics for the thresholds, calculate exponential the moving averages for tresholds.
    - Calculate information driven bars (Volume imbalance bars)

In [253]:
import json
import pandas as pd
import numpy as np
import os
from datetime import datetime
import asyncio
import nest_asyncio
import ipaddress
from bson import json_util
import pymongo
from datetime import datetime, timedelta
from statistics import mean, stdev

files_tickdata = os.listdir("Data")
files_tickdata = [f for f in files_tickdata if f != '.ipynb_checkpoints']
print(files_tickdata)
#print("\n")

['MatchTrades 2025-09-22 11:53:26 to 2025-09-22 18:48:18.json', 'MatchTrades 2025-09-23 09:47:21 to 2025-09-23 19:53:02.json', 'MatchTrades 2025-09-22 18:55:04 to 2025-09-23 09:35:06.json']


In [256]:
from pathlib import Path

## --------------------------------------------------------- ##
##  (1)                 get all the files in the folder      ##
## --------------------------------------------------------- ##

# sort data files from the latest to newest 
path = Path("Data")
files_sorted = sorted(
    path.iterdir(),
    key=lambda f: f.stat().st_mtime, 
    reverse=False                      
)

files_tickdata = []

for f in files_sorted:
    files_tickdata.append(f.name)
    
files_tickdata = [f for f in files_tickdata if f != '.ipynb_checkpoints']
print(files_tickdata)

## --------------------------------------------------------- ##
##  End (1)                                                  ##
## --------------------------------------------------------- ##

for file in files_tickdata:
    
    ## ----------------------------------------------------------- ##
    ##  (2)  Open files and Enforce Column Types through pandas    ##
    ## ----------------------------------------------------------- ##
    
    # Load the data
    # amount = float32
    # price = float64
    
    with open("Data/"+file) as f:
        MatchTrades_tickdata_df = pd.read_json(f).astype({"qty":"float32","price":"float64"})
        MatchTrades_tickdata_df["timestamp"] = pd.to_datetime(MatchTrades_tickdata_df["timestamp"], unit='ms')

    print(MatchTrades_tickdata_df)
    #MatchTrades_tickdata_df = MatchTrades_tickdata_df.iloc[0:1000][["timestamp","qty","price","isBuyerMaker","isBestMatch"]]
    
    ## ----------------------------------------------------------- ##
    ##     End (2)                                                 ##
    ## ----------------------------------------------------------- ##
    
    ## ------------------------------------------------- ##
    ##  (3)  Calculate standard bars (Tick bars)         ##
    ## ------------------------------------------------- ##
    
    ## Convert timestamp from milliseconds to datetime
    df = MatchTrades_tickdata_df
    
    # create the minute aggregated data
    # volume = float32
    # n_ticks = int16
    
    minute_tick_bars = (
        df
        .set_index('timestamp')
        .resample('1min')
        .agg({'qty': 'sum'})
        .rename(columns={'qty': 'volume'})
        .assign(n_ticks=lambda x: df.set_index('timestamp').resample('1min').size())
        .astype({
            "volume": "float32",
            "n_ticks": "int16"
        })
    ).reset_index()
    
    minute_tick_bars["timestamp"] = pd.to_datetime(minute_tick_bars["timestamp"], unit='s')
    
    ## calculate exponential the moving averages for tresholds
    # volume_per_min = float32
    # n_ticks_per_min = int16
    
    volume_per_min = (minute_tick_bars["volume"]).ewm(span=60,adjust=False).mean().round(5).astype("float64")
    n_ticks_per_min = (minute_tick_bars["n_ticks"]).ewm(span=60,adjust=False).mean().round(0).astype("int16")
    
    ## minute data for creating the tick bars
    df_ema = pd.DataFrame({
        "timestamp": minute_tick_bars["timestamp"],
        "volume_per_min": volume_per_min,
        "n_ticks_per_min": n_ticks_per_min
    })
    
    ## create dataframes for creating Tick  bars 
    TickBars_df = pd.DataFrame()
    df_ema_tick = df_ema

    T_Bar = T_bar()
    
    for row in MatchTrades_tickdata_df.itertuples(index=True, name="Trade"):
    
        timestamp = row.timestamp
        
        try:
            if timestamp.replace(second=0, microsecond=0) == df_ema.iloc[0]["timestamp"]:
                n_ticks_per_min_x = df_ema_tick.iloc[0]["n_ticks_per_min"]
                df_ema_tick = df_ema_tick.drop(0).reset_index(drop=True)
        except:
            pass

        if T_Bar.get_nticks() == None:
            T_Bar.initiate_bar(row)
        else:
            T_Bar.update(row)
                    
        if T_Bar.get_nticks() >= int(n_ticks_per_min_x/5):
            TickBar = T_Bar.create_bar()
            TickBars_df = pd.concat([TickBars_df, TickBar], ignore_index=True)

            T_Bar = T_bar()
            
    TickBars_df["timestamp"] =  TickBars_df["timestamp"].apply(lambda x: int(x.timestamp()* 1000))
    
    ## ----------------------------------------- ##
    ##  (3.1)  Save data                         ##
    ## ----------------------------------------- ##
    
    raw_data = TickBars_df.to_dict(orient="records")
    json_raw_data = json_util.dumps(raw_data)
    
    end_ts = float(TickBars_df.iloc[0]["timestamp"])/1000
    start_ts = float(TickBars_df.iloc[-1]["timestamp"])/1000
    
    with open("Bars/TickBars/"+"TickBars_df "+ datetime.fromtimestamp(end_ts).strftime("%Y-%m-%d %H:%M:%S") + " to " + datetime.fromtimestamp(start_ts).strftime("%Y-%m-%d %H:%M:%S") +".json", "w") as f:
                f.write(json_raw_data)
    
    ## ----------------------------------------- ##
    ##  (3.1)  End                               ##
    ## ----------------------------------------- ##

    ## ------------------------------------------------- ##
    ##  (3)  End                                         ##
    ## ------------------------------------------------- ##

    ## ------------------------------------------------- ##
    ##  (4)  Calculate standard bars (Volume bars)       ##
    ## ------------------------------------------------- ##

    ## create dataframes for creating Tick  bars 
    VolumeBars_df = pd.DataFrame()
    df_ema_volume = df_ema

    V_Bar = V_bar()
    
    for row in MatchTrades_tickdata_df.itertuples(index=True, name="Trade"):
    
        timestamp = row.timestamp
        
        try:
            if timestamp.replace(second=0, microsecond=0) == df_ema.iloc[0]["timestamp"]:
                volume_per_min_x = df_ema_volume.iloc[0]["volume_per_min"]
                df_ema_volume = df_ema_volume.drop(0).reset_index(drop=True)
        except:
            pass
    
        if V_Bar.get_total_volume() == None:
            V_Bar.initiate_bar(row)
        else:
            V_Bar.update(row)


        if V_Bar.get_total_volume() >= float(volume_per_min_x/5):
            VolumeBar = V_Bar.create_bar()
            VolumeBars_df = pd.concat([VolumeBars_df, VolumeBar], ignore_index=True)
    
            V_Bar = V_bar()

    VolumeBars_df["timestamp"] =  VolumeBars_df["timestamp"].apply(lambda x: int(x.timestamp()* 1000))
    
    ## ----------------------------------------- ##
    ##  (4.1)  Save data                         ##
    ## ----------------------------------------- ##
    
    raw_data = VolumeBars_df.to_dict(orient="records")
    json_raw_data = json_util.dumps(raw_data)
    
    end_ts = float(VolumeBars_df.iloc[0]["timestamp"])/1000
    start_ts = float(VolumeBars_df.iloc[-1]["timestamp"])/1000
    
    with open("Bars/VolumeBars/"+"VolumeBars_df "+ datetime.fromtimestamp(end_ts).strftime("%Y-%m-%d %H:%M:%S") + " to " + datetime.fromtimestamp(start_ts).strftime("%Y-%m-%d %H:%M:%S") +".json", "w") as f:
                f.write(json_raw_data)

    ## ------------------------------------------ ##
    ##  (4.1)  End                                ##
    ## ------------------------------------------ ##
    
    ## ------------------------------------------------- ##
    ##  (4)  End                                         ##
    ## ------------------------------------------------- ##

    ## ----------------------------------------------------------------------- ##
    ##  (5) Calculate information driven bars (Volume Imbalance bars)          ##
    ## ----------------------------------------------------------------------- ##

    ## create dataframes for creating Volume Imballance bars 
    btvt = np.array([]) # Tick data for the Tick Bar (TB)
    Imbalance_VolBars_df = pd.DataFrame()

    MatchTrades_tickdata_df['sign'] = tick_rule_sign(MatchTrades_tickdata_df['price'])
    MatchTrades_tickdata_df['signed_vol'] = MatchTrades_tickdata_df['sign'] * MatchTrades_tickdata_df['qty']
    
    ## initial values for the threshold (initilization)
    E0_T = 20
    E0_bv = 0.01

    VI_Bar = VI_bar()

    # maximum number of ticks in a bar
    rolling_q95 = 0
    max_ticks = 3*desired_ticks
    
    for row in MatchTrades_tickdata_df.itertuples(index=True, name="Trade"):

        if VI_Bar.get_total_signed_vol() == None:
            VI_Bar.initiate_bar(row)
        else:
            VI_Bar.update(row)
            
        #Tickdata_MatchTrades_IVB = pd.concat([Tickdata_MatchTrades_IVB, pd.DataFrame([row])], ignore_index=True)
        #signed_vol_total = Tickdata_MatchTrades_IVB["signed_vol"].astype(float).sum()
        btvt = np.append(btvt, row.signed_vol)      
        
        if (abs(VI_Bar.get_total_signed_vol()) >= int(E0_T)*abs(E0_bv)) or (VI_Bar.get_nticks() >= max_ticks):

            Imbalance_VolBar = VI_Bar.create_bar()

            # number of ticks in a bar
            # Imbalance_VolBar["E_T"] = len(Tickdata_MatchTrades_IVB)
            
            # v+ - v-
            Imbalance_VolBar["E_bv"] =  pd.Series(btvt).ewm(alpha=0.05, adjust=False).mean().iloc[-1]
            btvt = np.array([])
            
            Imbalance_VolBars_df = pd.concat([Imbalance_VolBars_df, Imbalance_VolBar], ignore_index=True)
            
            # recalculate treshold #
            E0_T = Imbalance_VolBars_df["E_T"].ewm(alpha=0.01, adjust=False).mean().iloc[-1] # estimate of E0[T]
            E0_bv = Imbalance_VolBars_df["E_bv"].ewm(alpha=0.01, adjust=False).mean().iloc[-1] # estimate of E0_bv

            # recalculate the new max nticks inside a bar:
            try:
                #UCL = mean(Imbalance_VolBars_df["E_T"]) + 3*stdev(Imbalance_VolBars_df["E_T"]) # upper confindence limit for the number of ticks in the bar
                rolling_q = mean(Imbalance_VolBars_df["E_T"]) + 2*stdev(Imbalance_VolBars_df["E_T"])
                max_ticks = int(rolling_q)
            except:
                max_ticks = 3*desired_ticks
            
            VI_Bar = VI_bar()

    Imbalance_VolBars_df["timestamp"] =  Imbalance_VolBars_df["timestamp"].apply(lambda x: int(x.timestamp()* 1000))
    
    ## ------------------------------------------------- ##
    ##  (5.1) save data                                  ##
    ## ------------------------------------------------- ##

    raw_data = Imbalance_VolBars_df.to_dict(orient="records")
    json_raw_data = json_util.dumps(raw_data)
    
    end_ts = float(Imbalance_VolBars_df.iloc[0]["timestamp"])/1000
    start_ts = float(Imbalance_VolBars_df.iloc[-1]["timestamp"])/1000
    
    with open("Bars/VolumeImbalanceBars/"+"Vol ImbalanceBars_df "+ datetime.fromtimestamp(end_ts).strftime("%Y-%m-%d %H:%M:%S") + " to " + datetime.fromtimestamp(start_ts).strftime("%Y-%m-%d %H:%M:%S") +".json", "w") as f:
                f.write(json_raw_data)
    
    ## ------------------------------------------------- ##
    ##  (5.1) End                                        ##
    ## ------------------------------------------------- ##
      
    ## ----------------------------------------------------------------------- ##
    ##  (5)  End                                                               ##
    ## ----------------------------------------------------------------------- ##
    

['MatchTrades 2025-09-22 11:53:26 to 2025-09-22 18:48:18.json', 'MatchTrades 2025-09-22 18:55:04 to 2025-09-23 09:35:06.json', 'MatchTrades 2025-09-23 09:47:21 to 2025-09-23 19:53:02.json']
                     timestamp      qty      price  isBuyerMaker  isBestMatch
0      2025-09-22 10:53:26.880  0.00001  112777.64         False         True
1      2025-09-22 10:53:26.880  0.01602  112777.64         False         True
2      2025-09-22 10:53:26.880  0.00670  112777.64         False         True
3      2025-09-22 10:53:27.424  0.00216  112777.64         False         True
4      2025-09-22 10:53:27.424  0.00019  112777.64         False         True
...                        ...      ...        ...           ...          ...
819967 2025-09-22 17:48:17.913  0.00005  112468.52         False         True
819968 2025-09-22 17:48:17.913  0.00005  112468.52         False         True
819969 2025-09-22 17:48:17.913  0.00005  112468.52         False         True
819970 2025-09-22 17:48:17.913

# (3) Feature Preparation

## This code is used to:
- Calculate the properties from Tick bars:
    - Relative strength index (RSI).
    - Average true range (ATR).
    - Average directional index (ADI).
    - Simple moving averages (SMA).
    - Exponential moving averages (EMA).
- Create signals for entry and exiting the market:
    - Simple moving averages (SMA) (9,21).
    - Exponential moving averages (EMA) (9,21).
    - Relative strength index (RSI) (30,70).

In [258]:
from pathlib import Path

folders = ["TickBars","VolumeBars","VolumeImbalanceBars"]

for folder in folders:

    path = Path(f"Bars/{folder}")
    files_sorted = sorted(
        path.iterdir(),
        key=lambda f: f.stat().st_mtime, 
        reverse=False                      
    )
    
    files_bars = []

    for f in files_sorted:
        files_bars.append(f.name)
    
    files_bars = [f for f in files_bars if f != '.ipynb_checkpoints']

    for file in files_bars:

        # Load the data
        with open(f"Bars/{folder}/"+file) as f:
            TickBars_df = pd.read_json(f)
            TickBars_df["timestamp"] = pd.to_datetime(TickBars_df["timestamp"], unit='ms')
            
        # calculating the predictors for 
        calc_pred = predictors(TickBars_df)
        pred_df = pd.DataFrame(calc_pred.get_predictors())
        pred_df["timestamp"] =  pred_df["timestamp"].apply(lambda x: int(x.timestamp()* 1000))
        
        ##########################################
        ##              Save data               ##
        
        raw_data = pred_df.to_dict(orient="records")
        json_raw_data = json_util.dumps(raw_data)
        
        end_ts = float(pred_df.iloc[0]["timestamp"])/1000
        start_ts = float(pred_df.iloc[-1]["timestamp"])/1000
        
        with open(f"Features/{folder} Features/"+f"{folder} Features df "+ datetime.fromtimestamp(end_ts).strftime("%Y-%m-%d %H:%M:%S") + " to " + datetime.fromtimestamp(start_ts).strftime("%Y-%m-%d %H:%M:%S") +".json", "w") as f:
                    f.write(json_raw_data)
        
        ##########################################
        ##########################################
        
        # removing the firsts rows with Nan values
        signal_df = pred_df.iloc[26:]
    
        # -------------------------------------------------------------------------------- #
        #             creating a signal to buy and sell from the predictors                #
        # -------------------------------------------------------------------------------- #
        
    
        # SMA 9,21 signals
        
        # 1 = SMA 9 crosses above SMA 21 (bullish)
        signal_df['Signal SMA'] = ((signal_df['SMA 9'] > signal_df['SMA 21']) & 
                       (signal_df['SMA 9'].shift(1) <= signal_df['SMA 21'].shift(1))).astype(int)
        
        # -1 = SMA 9 crosses below SMA 21 (bearish)
        signal_df['Signal SMA'] -= ((signal_df['SMA 9'] < signal_df['SMA 21']) & 
                        (signal_df['SMA 9'].shift(1) >= signal_df['SMA 21'].shift(1))).astype(int)
        
        # Bullish crossover: EMA 9 crosses ABOVE EMA 21
        signal_df.loc[
            (signal_df['EMA 9'] > signal_df['EMA 21']) & 
            (signal_df['EMA 9'].shift(1) <= signal_df['EMA 21'].shift(1)),
            "Signal EMA"
        ] = 1
        
        # Bearish crossover: EMA 9 crosses BELOW EMA 21
        signal_df.loc[
            (signal_df['EMA 9'] < signal_df['EMA 21']) & 
            (signal_df['EMA 9'].shift(1) >= signal_df['EMA 21'].shift(1)),
            "Signal EMA"
        ] = -1
        
        # Fill the rest with 0 (no signal)
        signal_df["Signal EMA"] = signal_df["Signal EMA"].fillna(0).astype(int)
        
        # Generate signals for RSI
        signal_df['Signal RSI'] = np.where(signal_df['RSI'] < 30, 1, np.where(signal_df['RSI'] > 70, -1, 0))
        
        ##########################################
        ##          Save signals data           ##
        
        raw_data = signal_df.to_dict(orient="records")
        json_raw_data = json_util.dumps(raw_data)
        
        end_ts = float(signal_df.iloc[0]["timestamp"])/1000
        start_ts = float(signal_df.iloc[-1]["timestamp"])/1000
        
        with open(f"Signals/{folder} Signals/"+f"{folder} Signals df "+ datetime.fromtimestamp(end_ts).strftime("%Y-%m-%d %H:%M:%S") + " to " + datetime.fromtimestamp(start_ts).strftime("%Y-%m-%d %H:%M:%S") +".json", "w") as f:
                    f.write(json_raw_data)
    
        ##########################################
        ##########################################
        
        # -------------------------------------------------------------------------------- #
        #                                                                                  #
        # -------------------------------------------------------------------------------- #


['TickBars_df 2025-09-22 11:53:26 to 2025-09-22 18:48:06.json', 'TickBars_df 2025-09-22 18:55:04 to 2025-09-23 09:34:05.json', 'TickBars_df 2025-09-23 09:47:25 to 2025-09-23 19:53:01.json']


/tmp/ipykernel_9226/1421446778.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  signal_df['Signal SMA'] = ((signal_df['SMA 9'] > signal_df['SMA 21']) &
/tmp/ipykernel_9226/1421446778.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  signal_df['Signal SMA'] -= ((signal_df['SMA 9'] < signal_df['SMA 21']) &
/tmp/ipykernel_9226/1421446778.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

['VolumeBars_df 2025-09-22 11:53:26 to 2025-09-22 18:47:52.json', 'VolumeBars_df 2025-09-22 18:55:04 to 2025-09-23 09:34:05.json', 'VolumeBars_df 2025-09-23 09:47:25 to 2025-09-23 19:52:50.json']


/tmp/ipykernel_9226/1421446778.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  signal_df['Signal SMA'] = ((signal_df['SMA 9'] > signal_df['SMA 21']) &
/tmp/ipykernel_9226/1421446778.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  signal_df['Signal SMA'] -= ((signal_df['SMA 9'] < signal_df['SMA 21']) &
/tmp/ipykernel_9226/1421446778.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

['Vol ImbalanceBars_df 2025-09-22 11:53:26 to 2025-09-22 18:46:55.json', 'Vol ImbalanceBars_df 2025-09-22 18:55:04 to 2025-09-23 09:34:05.json', 'Vol ImbalanceBars_df 2025-09-23 09:47:25 to 2025-09-23 19:53:01.json']


/tmp/ipykernel_9226/1421446778.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  signal_df['Signal SMA'] = ((signal_df['SMA 9'] > signal_df['SMA 21']) &
/tmp/ipykernel_9226/1421446778.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  signal_df['Signal SMA'] -= ((signal_df['SMA 9'] < signal_df['SMA 21']) &
/tmp/ipykernel_9226/1421446778.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave